## SLT Trainer

#### DataLoader

In [ ]:
import sys
sys.path.insert(0, './Dataloader')
from SLT_dataloader import EEGROIDataset, SignalDataCollator, EEGROI_fft_Dataset, EEGROI_Power_Dataset

# Usage example
roi_folder = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca"
eeg_folder = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca"
group_file = "./Dataloader/subject_groups.json"
# segment_file = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\RO\I\Desikan_Kilianny_with_3pca\\roi_removal_segment.txt"

# Create dataset
train_dataset = EEGROI_fft_Dataset(roi_folder, eeg_folder, group_file, "train_dataset" )
print(f"\n\n================================================\n")
print(f"Total dataset size: {len(train_dataset)}")
print(f"\n================================================\n\n")
# Create dataset
test_dataset = EEGROI_fft_Dataset(roi_folder, eeg_folder, group_file, "test_eval")
print(f"\n================================================\n")
print(f"Total eval dataset size: {len(test_dataset)}")

### Simulate Dataset

In [ ]:
import sys
sys.path.insert(0, './Dataloader')
from SLT_dataloader import EEGROIDataset, SignalDataCollator, EEGROI_fft_Dataset, EEGROI_Power_Dataset

train_path = "G:\\共用雲端硬碟\CNElab_陳昱祺\\source localization\\simulate_data\\training_data\\"    # dataset_seedsource_5000_1000000.0_20250402_065238
test_path =   "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca\\"
group_file = "./Dataloader/subject_groups.json"

# Create dataset
train_dataset = EEGROI_Power_Dataset(train_path)
print(f"\n\n================================================\n")
print(f"Total dataset size: {len(train_dataset)}")
print(f"\n================================================\n\n")
# Create dataset
test_dataset = EEGROI_fft_Dataset(test_path, test_path, group_file, "train_dataset")
print(f"\n================================================\n")
print(f"Total eval dataset size: {len(test_dataset)}")

### Merge Dataset

In [ ]:
import sys
sys.path.insert(0, './Dataloader')
from SLT_dataloader import EEGROIDataset, SignalDataCollator, EEGROI_fft_Dataset, EEGROI_Power_Dataset, EEGROI_Merge_Dataset

data_path_1 = "C:\\Users\\user\\Desktop\\Source Localization\\Source-Localization-Simulation\\simulate_data\\dataset_seedsource_100tp1000_2000_10_20250428_174623\\"
data_path_2 = "C:\\Users\\user\\Desktop\\Source Localization\\Source-Localization-Simulation\\simulate_data\\dataset_seedsource_100to1000_2000_1000000.0_20250428_174549\\"
data_path_3 = "C:\\Users\\user\\Desktop\\Source Localization\\Source-Localization-Simulation\\simulate_data\\dataset_seedsource_100to1000_2000_1_20250428_174640\\"
data_path_4 = "C:\\Users\\user\\Desktop\\Source Localization\\Source-Localization-Simulation\\simulate_data\\dataset_seedsource_2to100_2000_10_20250428_201233\\"
data_path_5 = "C:\\Users\\user\\Desktop\\Source Localization\\Source-Localization-Simulation\\simulate_data\\dataset_seedsource_2to100_2000_1000000.0_20250428_201134\\"
test_path = "C:\\Users\\user\\Desktop\\Source Localization\\Desikan_Kilianny_with_3pca\\"
group_file = "./Dataloader/subject_groups.json"

dataset_1 = EEGROI_Power_Dataset(data_path_1)
dataset_2 = EEGROI_Power_Dataset(data_path_2)
dataset_3 = EEGROI_Power_Dataset(data_path_3)
dataset_4 = EEGROI_Power_Dataset(data_path_2)
dataset_5 = EEGROI_Power_Dataset(data_path_3)
dataset_6 = EEGROI_fft_Dataset(test_path, test_path, group_file, "train_dataset")

merge_dataset = EEGROI_Merge_Dataset([dataset_1, dataset_2, dataset_3, dataset_4, dataset_5, dataset_6])
train_data_len = len(merge_dataset)
print(f"\n\n================================================\n")
print(f"Total dataset size: {train_data_len}")
print(f"\n================================================\n\n")

test_dataset_1 = EEGROI_fft_Dataset(test_path, test_path, group_file, "test_dataset_temp")
test_data_path = "C:\\Users\\user\\Desktop\\Source Localization\\Source-Localization-Simulation\\simulate_data\\dataset_seedsource_2to100_2000_1_20250429_144003\\"
test_dataset_2 = EEGROI_Power_Dataset(test_data_path)
merge_eval_dataset = EEGROI_Merge_Dataset([test_dataset_1, test_dataset_2])
test_data_len = len(merge_eval_dataset)
print(f"\n================================================\n")
print(f"Total eval dataset size: {test_data_len}")

EEG Power shape: torch.Size([20000, 30, 100])
Source Power shape: torch.Size([20000, 204, 100])
EEG Power shape: torch.Size([20000, 30, 100])
Source Power shape: torch.Size([20000, 204, 100])
EEG Power shape: torch.Size([20000, 30, 100])
Source Power shape: torch.Size([20000, 204, 100])
EEG Power shape: torch.Size([20000, 30, 100])
Source Power shape: torch.Size([20000, 204, 100])
EEG Power shape: torch.Size([20000, 30, 100])
Source Power shape: torch.Size([20000, 204, 100])
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', 

#### Model

In [ ]:
import sys
sys.path.insert(0, './FirstMultiModel/EEGART')
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from SLT_dataloader import SignalDataCollator, RandonMaskDataCollator

from tf_config import SLTConfig
from tf_model import SLTModel, SLTModel_ver2
from datetime import datetime

# 获取当前日期时间，并格式化为 YYYYMMDD
current_date = datetime.now().strftime("%Y%m%d_%H%M%S")

# 替换路径中的日期
runname = "mask_prob_08_tgtwithoutEEG_learnable_position_"
output_dir = f"./results_3pca/{runname}_{current_date}"

# 假設你的設備是 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

"""
    Parameters:
    
"""
# src_channel_size = 30
# tgt_channel_size = 204
# N = 4
# source_voxel_time = 51
# sensor_time = 51
# tgt_d_model = 51
# slt_config_fname = "test_slt_confit"

# 包裝自定義的初始化方法
def huggingface_model_init():
    
    src_len = 30
    tgt_len = 204 # atlas size 
    N = 6
    source_voxel_time = 100
    sensor_time = 100
    tgt_d_model = 512
    slt_config_fname = "test_slt_confit"
    
    slt_config = SLTConfig(src_channel_size=src_len, tgt_channel_size=tgt_len, N=N,
                           source_voxel_time=source_voxel_time, sensor_time=sensor_time, tgt_d_model=tgt_d_model)
    slt_config.save_pretrained(slt_config_fname)
    slt_model = SLTModel(slt_config)
    slt_model = slt_model.to(device)
    for p in slt_model.parameters():
        if p.dim() > 1:
            torch.nn.init.xavier_uniform_(p)
    return slt_model

batch_size = 256
training_args = TrainingArguments(
    output_dir=output_dir,
    run_name = runname,
    eval_strategy="steps",
    per_device_train_batch_size=batch_size,   # training batch size
    per_device_eval_batch_size=batch_size,    # test batch size
    eval_steps= int((train_data_len/batch_size)/3),  # eval steps= 1/3 steps of one epoch
    eval_accumulation_steps= 4,        
    num_train_epochs=100,              # epoch
    weight_decay=0.01,                 # 
    learning_rate = 0.0005,
    lr_scheduler_type = "constant",
    # fp16=True,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps= int((train_data_len/batch_size)/10),
    report_to="wandb",
    dataloader_num_workers=8,  # 🚀 Use multiple workers
    dataloader_pin_memory=True,  # 🚀 Reduce CPU-GPU data transfer time
)

# def compute_metrics(eval_preds, batch_size=64):
#     predictions, targets = eval_preds
    
#     loss_fct = nn.MSELoss()
#     num_samples = predictions.shape[0]
#     mse_sum = 0.0
#     num_batches = 0

#     for start_idx in range(0, num_samples, batch_size):
#         end_idx = min(start_idx + batch_size, num_samples)
#         logits_batch = predictions[start_idx:end_idx]
#         labels_batch = targets[start_idx:end_idx]
#         logits = torch.tensor(logits_batch, dtype=torch.float32)
#         labels = torch.tensor(labels_batch, dtype=torch.float32)
#         # Compute the z-scores for the batch
#         logits_mean = torch.mean(logits, dim=(1, 2), keepdim=True)
#         logits_std = torch.std(logits, dim=(1, 2), keepdim=True)
#         logits_norm = (logits - logits_mean) / (logits_std)
        
#         labels_mean = torch.mean(labels, dim=(1, 2), keepdim=True)
#         labels_std = torch.std(labels,  dim=(1, 2), keepdim=True)
#         labels_norm = (labels - labels_mean) / (labels_std)
        
#         # Compute batch MSE
#         mse_sum += loss_fct(logits_norm, labels_norm).item()
#         num_batches += 1

#     # Return the average MSE
#     avg_mse = mse_sum / num_batches
    
#     del logits, labels, logits_norm, labels_norm
#     torch.cuda.empty_cache()
    
#     return {"mse": avg_mse}

def dummy_compute_metrics(eval_preds):
    return {}  # Return empty dict

# 初始化模型和 Trainer
trainer = Trainer(
    # model=slt_model,
    model_init=huggingface_model_init,
    args=training_args,
    train_dataset=merge_dataset,
    eval_dataset=merge_eval_dataset,
    data_collator=RandonMaskDataCollator(mask_prob=0.50),
    # compute_metrics=dummy_compute_metrics,
)

# 开始训练
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: richielp700 (richielp700-national-tsing-hua-university). Use `wandb login --relogin` to force relogin


  0%|          | 0/48500 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.3294, 'grad_norm': 0.14074890315532684, 'learning_rate': 0.0005, 'epoch': 0.1}
{'loss': 1.1341, 'grad_norm': 0.08792014420032501, 'learning_rate': 0.0005, 'epoch': 0.2}
{'loss': 1.1152, 'grad_norm': 0.15132112801074982, 'learning_rate': 0.0005, 'epoch': 0.3}


  0%|          | 0/141 [00:00<?, ?it/s]

...

{'eval_loss': 0.250173419713974, 'eval_runtime': 78.1566, 'eval_samples_per_second': 459.744, 'eval_steps_per_second': 1.804, 'epoch': 87.97}
{'loss': 0.5802, 'grad_norm': 0.128443643450737, 'learning_rate': 0.0005, 'epoch': 87.98}
{'loss': 0.5797, 'grad_norm': 0.13510838150978088, 'learning_rate': 0.0005, 'epoch': 88.08}
{'loss': 0.5817, 'grad_norm': 0.12889033555984497, 'learning_rate': 0.0005, 'epoch': 88.18}
{'loss': 0.5781, 'grad_norm': 0.11937539279460907, 'learning_rate': 0.0005, 'epoch': 88.28}


  0%|          | 0/141 [00:00<?, ?it/s]

{'eval_loss': 0.24979804456233978, 'eval_runtime': 78.3274, 'eval_samples_per_second': 458.741, 'eval_steps_per_second': 1.8, 'epoch': 88.3}
{'loss': 0.5793, 'grad_norm': 0.14908108115196228, 'learning_rate': 0.0005, 'epoch': 88.38}
{'loss': 0.5789, 'grad_norm': 0.15318351984024048, 'learning_rate': 0.0005, 'epoch': 88.48}
{'loss': 0.5778, 'grad_norm': 0.11701368540525436, 'learning_rate': 0.0005, 'epoch': 88.58}


  0%|          | 0/141 [00:00<?, ?it/s]

{'eval_loss': 0.24824146926403046, 'eval_runtime': 78.3498, 'eval_samples_per_second': 458.61, 'eval_steps_per_second': 1.8, 'epoch': 88.63}
{'loss': 0.5798, 'grad_norm': 0.18549005687236786, 'learning_rate': 0.0005, 'epoch': 88.68}
{'loss': 0.5763, 'grad_norm': 0.14649957418441772, 'learning_rate': 0.0005, 'epoch': 88.78}
{'loss': 0.5762, 'grad_norm': 0.14730645716190338, 'learning_rate': 0.0005, 'epoch': 88.87}
